This script loads behavioral mice data (from `biasedChoiceWorld` protocol and, separately, the last three sessions of training) only from mice that pass a given (stricter) training criterion. For the `biasedChoiceWorld` protocol, only sessions achieving the `trained_1b` and `ready4ephysrig` training status are collected.
The data are slightly reformatted and saved as `.csv` files.

In [1]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults, SessionTrainingStatus
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavior_analysis

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Connecting lacerbi@datajoint.internationalbrainlab.org:3306


In [2]:
import os
myPath = r"C:\Users\Luigi\Documents\GitHub\ibl-changepoint\data" # Write here your data path
os.chdir(myPath)

In [6]:
# Define mice of interest
mice_names = ['test_human']
print(mice_names)

['test_human']


In [20]:
sess = (
    (acquisition.Session * behavior_analysis.SessionTrainingStatus) 
        # & 'task_protocol LIKE "%biased%"' & 'session_start_time < "2019-09-30"' 
        # & ('training_status="trained_1b" OR training_status="ready4ephysrig"')
    )
              
saved_mice_names = list()

# Perform at least this number of sessions
MinSessionNumber = 1

def get_mouse_data(df):
    position_deg = 35.    # Stimuli appear at +/- 35 degrees

    # Create new dataframe
    datamat = pd.DataFrame()
    datamat['trial_num'] = df['trial_id']
    datamat['session_num'] = np.cumsum(df['trial_id'] == 1)
    datamat['stim_probability_left'] = df['trial_stim_prob_left']
    signed_contrast = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
    datamat['contrast'] = np.abs(signed_contrast)
    datamat['position'] = np.sign(signed_contrast)*position_deg
    datamat['response_choice'] = df['trial_response_choice']
    datamat.loc[df['trial_response_choice'] == 'CCW','response_choice'] = 1
    datamat.loc[df['trial_response_choice'] == 'CW','response_choice'] = -1
    datamat.loc[df['trial_response_choice'] == 'No Go','response_choice'] = 0
    datamat['trial_correct'] = np.double(df['trial_feedback_type']==1)
    datamat['reaction_time'] = df['trial_response_time'] - df['trial_stim_on_time'] # double-check

    # Since some trials have zero contrast, need to compute the alleged position separately
    datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'response_choice']*position_deg
    datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'response_choice']*(-position_deg)
    
    return datamat

In [23]:
# Loop over all mice
for mouse_nickname in mice_names:
    
    mouse_subject = {'subject_nickname': mouse_nickname}
    
    # Get data for biased sessions
    behavior_save = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
        * sess.proj('session_uuid','task_protocol','session_start_time','training_status') * subject.Subject.proj('subject_nickname') \
        * subject.SubjectLab.proj('lab_name')
    df = pd.DataFrame(behavior_save.fetch(order_by='subject_nickname, session_start_time, trial_id', as_dict=True))

    if len(df) > 0:           # The mouse has performed in at least one stable session with biased blocks            
        datamat = get_mouse_data(df)

        # Take mice that have performed a minimum number of sessions
        if np.max(datamat['session_num']) >= MinSessionNumber:

            # Should add 'N' to mice names that start with numbers?

            # Save dataframe to CSV file
            filename = mouse_nickname + '.csv'
            datamat.to_csv(filename,index=False)
            saved_mice_names.append(mouse_nickname)            

print(saved_mice_names)

['test_human', 'test_human']


In [22]:
df

,subject_uuid,session_start_time,trial_id,trial_start_time,trial_end_time,trial_response_time,trial_response_choice,trial_stim_on_time,trial_stim_contrast_left,trial_stim_contrast_right,...,trial_rep_num,trial_stim_prob_left,trial_reward_volume,trial_iti_duration,trial_included,session_uuid,task_protocol,training_status,subject_nickname,lab_name
0,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-14 10:52:51,1,0.0000,12.354102,10.8541,CW,0.6156,0.0,0.000,...,None,0.2,None,None,None,1a88ee0d-f67f-4eb7-84f6-88bc757fd17e,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
1,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-14 10:52:51,2,12.8201,14.666602,13.9665,CCW,13.4966,0.0,0.250,...,None,0.2,None,None,None,1a88ee0d-f67f-4eb7-84f6-88bc757fd17e,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
2,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-14 10:52:51,3,15.0709,17.131402,16.4313,CCW,15.7146,0.0,0.125,...,None,0.2,None,None,None,1a88ee0d-f67f-4eb7-84f6-88bc757fd17e,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
3,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-14 10:52:51,4,17.4797,20.184002,19.4839,CW,18.1296,0.0,0.000,...,None,0.2,None,None,None,1a88ee0d-f67f-4eb7-84f6-88bc757fd17e,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
4,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-14 10:52:51,5,20.5524,22.350202,21.6501,CCW,21.2815,0.0,0.125,...,None,0.2,None,None,None,1a88ee0d-f67f-4eb7-84f6-88bc757fd17e,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-15 18:55:50,156,222.7167,224.401802,223.3920,CW,223.0486,0.0,0.000,...,None,0.2,None,None,None,55742c1a-00c1-4bb8-a450-aa403aaf457b,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
556,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-15 18:55:50,157,224.8242,225.821303,225.6110,CCW,225.3298,0.0,0.250,...,None,0.2,None,None,None,55742c1a-00c1-4bb8-a450-aa403aaf457b,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
557,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-15 18:55:50,158,226.3692,227.167802,226.9580,CCW,226.6828,0.0,0.000,...,None,0.2,None,None,None,55742c1a-00c1-4bb8-a450-aa403aaf457b,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab
558,4372e7ab-0978-4aff-81eb-e922e8b69920,2019-10-15 18:55:50,159,227.5867,228.405402,228.1950,CCW,227.9148,0.0,0.250,...,None,0.2,None,None,None,55742c1a-00c1-4bb8-a450-aa403aaf457b,_iblrig_tasks_biasedChoiceWorld6.0.5,in_training,test_human,churchlandlab


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,